In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os


# Explicación del Código en Python

Este bloque de código obtiene la ruta actual del script en ejecución y la modifica para apuntar a un directorio específico. A continuación, se detalla su funcionamiento.

### Código
```python
try:
    carpeta_actual = os.path.dirname(os.path.abspath(__file__))
except NameError:
    carpeta_actual = os.getcwd()  # Alternativa para entornos interactivos como Jupyter

Carpeta_de_entrenamiento = carpeta_actual.replace('creacion_red_neuronal_y_entrenamiento', 'datos_iniciales_del_sistema')
print(f"Carpeta actual: {Carpeta_de_entrenamiento}")


In [15]:
try:
    carpeta_actual = os.path.dirname(os.path.abspath(__file__))
except NameError:
    carpeta_actual = os.getcwd()  # Alternativa para entornos interactivos como Jupyter

Carpeta_de_entrenamiento=carpeta_actual.replace('creacion_red_neuronal_y_entrenamiento', 'datos_iniciales_del_sistema')
print(f"Carpeta actual: {Carpeta_de_entrenamiento}")



Carpeta actual: c:\Users\dgome\OneDrive\SIN RECUPERAR\Sql\Documentos\SistemaIot\Periferico\modelos_entrenamiento\vehiculo_autonomo\datos_iniciales_del_sistema


# Explicación del Código en Python

Este bloque de código construye una ruta para un archivo CSV en función de un directorio base, verifica si el archivo existe y luego intenta cargarlo usando la biblioteca `pandas`. Si ocurre algún error, el programa lo notifica y detiene su ejecución.




In [16]:

ruta_csv = os.path.join(Carpeta_de_entrenamiento,'tinyml_training_data_clean.csv')
# 3. Cargar el archivo CSV
if os.path.exists(ruta_csv):
    try:
        data = pd.read_csv(ruta_csv)
        print(f"Archivo CSV cargado correctamente. Filas: {data.shape[0]}, Columnas: {data.shape[1]}")
    except Exception as e:
        print(f"Error al cargar el archivo CSV: {e}")
        exit()
else:
    print(f"El archivo CSV no existe: {ruta_csv}")
    exit()


Archivo CSV cargado correctamente. Filas: 3737, Columnas: 8


# Explicación del Código en Python

Este bloque de código carga un archivo CSV y verifica que las columnas necesarias para realizar operaciones posteriores estén presentes. Si alguna columna está ausente, se notifica al usuario y se detiene la ejecución del programa.



In [17]:
data = pd.read_csv(ruta_csv)
# Verificar la existencia de las columnas necesarias
columnas_entrada = ['distancia_izquierda', 'distancia_derecha', 'distancia_izquierda_duration', 'distancia_derecha_duration']
columnas_salida = ['IN1', 'IN2', 'IN3', 'IN4']

for columna in columnas_entrada + columnas_salida:
    if columna not in data.columns:
        print(f"La columna '{columna}' no está presente en el archivo CSV.")
        exit()


# Explicación 

Este bloque de código toma los datos del archivo CSV, selecciona las columnas necesarias para el análisis, verifica si hay problemas en los datos (como valores faltantes), y prepara los datos de entrada para que sean consistentes y fáciles de trabajar en el modelo.


#### 1. Definir las Entradas y Salidas
```python
X = data[columnas_entrada].values
y = data[columnas_salida].values
```
- **¿Qué significa esto?**
  - Se separan los datos en dos grupos:
    - **Entradas (`X`)**: La información que usamos para hacer predicciones o análisis (por ejemplo, mediciones o características de un sistema).
    - **Salidas (`y`)**: Los resultados esperados o etiquetas que queremos predecir (por ejemplo, activaciones de un sistema o valores esperados).
  - Los datos se seleccionan de las columnas específicas del archivo CSV, indicadas en las listas `columnas_entrada` y `columnas_salida`.

#### 2. Validar si Hay Datos Faltantes
```python
if np.any(np.isnan(X)) or np.any(np.isnan(y)):
    print("Existen valores NaN en los datos de entrada o salida. Por favor, limpie los datos.")
    exit()
```
- **¿Qué hace esto?**
  - Revisa si hay valores faltantes (llamados "NaN", que significa "Not a Number") en las entradas o salidas.
  - Si encuentra datos faltantes:
    - Muestra un mensaje indicando el problema: "Existen valores NaN en los datos de entrada o salida. Por favor, limpie los datos."
    - Detiene la ejecución del programa.
- **¿Por qué es importante?**
  - Los valores faltantes pueden causar errores graves durante el análisis o entrenamiento del modelo, por lo que deben ser eliminados o corregidos antes de continuar.

#### 3. Normalizar las Entradas
```python
X_max = np.max(X, axis=0)
if np.any(X_max == 0):
    print("Error: Algunos valores máximos de las columnas de entrada son 0, lo que causará división por cero.")
    exit()
X = X / X_max
```
- **¿Qué significa "normalizar"?**
  - Es un proceso para convertir los datos en un rango más uniforme (en este caso, de 0 a 1).
  - Esto se logra dividiendo cada valor de entrada por el valor máximo de su columna.
- **¿Qué hace el código?**
  - Encuentra el valor más alto (`X_max`) en cada columna de las entradas.
  - Verifica si alguno de estos valores máximos es 0:
    - Si es así, muestra un error: "Algunos valores máximos de las columnas de entrada son 0, lo que causará división por cero."
    - Detiene la ejecución.
  - Si todo está bien, divide cada valor por su máximo correspondiente, haciendo que todos los datos estén en el rango 0-1.
- **¿Por qué es importante?**
  - Esto asegura que los datos sean consistentes y evita problemas matemáticos (como división por cero) que podrían ocurrir durante el análisis o entrenamiento del modelo.

### Resumen General
Este código:
1. **Separa** los datos en entradas (`X`) y salidas (`y`) para un modelo.
2. **Verifica** que no haya valores faltantes en los datos, ya que estos podrían causar problemas.
3. **Prepara** los datos de entrada normalizándolos, asegurando que estén en un rango uniforme y evitando errores matemáticos.

### Ejemplo para Entender Mejor
- Si tus datos de entrada son:
  ```
  distancia_izquierda: [10, 20, 30]
  distancia_derecha: [5, 10, 15]
  ```
  Después de normalizar, se convierten en:
  ```
  distancia_izquierda (normalizada): [0.33, 0.67, 1]
  distancia_derecha (normalizada): [0.33, 0.67, 1]
  ```
  Esto ayuda a que el modelo trabaje con datos consistentes sin dar más peso a valores más grandes de manera injusta.

In [18]:
# Definir las entradas (X) y las salidas (y)
X = data[columnas_entrada].values
y = data[columnas_salida].values

# Validar si hay datos faltantes
if np.any(np.isnan(X)) or np.any(np.isnan(y)):
    print("Existen valores NaN en los datos de entrada o salida. Por favor, limpie los datos.")
    exit()

# Normalizar las entradas para evitar errores de división por cero
X_max = np.max(X, axis=0)
if np.any(X_max == 0):
    print("Error: Algunos valores máximos de las columnas de entrada son 0, lo que causará división por cero.")
    exit()
X = X / X_max  # Normalizar de 0 a 1


# Explicación 

Este bloque de código divide los datos disponibles en dos grupos: uno para entrenar el modelo y otro para probarlo. Esto asegura que el modelo pueda ser evaluado de manera justa con datos que no ha visto antes.


#### 1. Dividir los Datos en Entrenamiento y Prueba
```python
try:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Datos divididos en entrenamiento ({X_train.shape[0]}) y prueba ({X_test.shape[0]}).")
except Exception as e:
    print(f"Error al dividir los datos de entrenamiento y prueba: {e}")
    exit()
```
- **¿Qué significa esto?**
  - Divide las entradas (`X`) y salidas (`y`) en dos partes:
    - **Entrenamiento** (`X_train`, `y_train`): Datos utilizados para construir el modelo.
    - **Prueba** (`X_test`, `y_test`): Datos utilizados para evaluar qué tan bien funciona el modelo.
  - El tamaño de los datos de prueba se define con `test_size=0.2`, lo que significa que el 20% de los datos se usará para pruebas y el 80% para entrenamiento.

#### 2. Parámetros Importantes
- **`test_size=0.2`**:
  - Proporción de datos asignados al conjunto de prueba.
  - En este caso, el 20% de los datos se destina a prueba y el 80% a entrenamiento.
- **`random_state=42`**:
  - Asegura que la división de los datos sea reproducible, es decir, siempre se obtendrán los mismos conjuntos de entrenamiento y prueba.
- **`X_train.shape[0]` y `X_test.shape[0]`**:
  - Devuelven el número de filas (ejemplos) en los conjuntos de entrenamiento y prueba respectivamente.

#### 3. Manejar Errores
```python
except Exception as e:
    print(f"Error al dividir los datos de entrenamiento y prueba: {e}")
    exit()
```
- **¿Qué hace esto?**
  - Si ocurre algún problema durante la división de los datos (por ejemplo, si las entradas y salidas no tienen el mismo tamaño), muestra un mensaje de error y detiene el programa.

### Propósito General
Este código asegura que:
1. Los datos se dividan correctamente para construir y evaluar el modelo.
2. Se mantenga una proporción fija entre los datos de entrenamiento y prueba.
3. Se manejen errores potenciales que podrían surgir debido a problemas con los datos.

### Ejemplo de Salida
1. **División Exitosa**:
   ```
   Datos divididos en entrenamiento (80) y prueba (20).
   ```
2. **Error Durante la División**:
   ```
   Error al dividir los datos de entrenamiento y prueba: Las dimensiones de X e y no coinciden.
   ```

Esto facilita tanto el entrenamiento como la evaluación del modelo al garantizar que las etapas estén separadas y que el conjunto de prueba no interfiera con el proceso de entrenamiento.


In [19]:
# Dividir los datos en conjuntos de entrenamiento y prueba
try:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Datos divididos en entrenamiento ({X_train.shape[0]}) y prueba ({X_test.shape[0]}).")
except Exception as e:
    print(f"Error al dividir los datos de entrenamiento y prueba: {e}")
    exit()

Datos divididos en entrenamiento (2989) y prueba (748).


# Explicación 

Este bloque de código crea un modelo de red neuronal utilizando la biblioteca TensorFlow y Keras. Este modelo está diseñado para procesar datos con un número específico de entradas y generar cuatro resultados como salida.

### Código Explicado

#### 1. Crear el Modelo
```python
try:
    input_shape = (X_train.shape[1],)  # Determinar automáticamente el número de características de entrada
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=input_shape),        # Entrada con la forma del dataset
        tf.keras.layers.Dense(8, activation='relu'),  # Capa oculta con 8 neuronas
        tf.keras.layers.Dense(4, activation='sigmoid') # Cuatro salidas (para IN1, IN2, IN3, IN4)
    ])
except Exception as e:
    print(f"Error al crear el modelo de la red neuronal: {e}")
    exit()
```
- **¿Qué significa esto?**
  - Se crea una red neuronal con tres capas:
    1. **Capa de entrada**: Toma datos con una cantidad específica de características (determinada automáticamente por `X_train.shape[1]`).
    2. **Capa oculta**: Tiene 8 "neuronas" y utiliza la función de activación **ReLU** (Rectified Linear Unit), que ayuda a capturar relaciones complejas en los datos.
    3. **Capa de salida**: Tiene 4 neuronas y utiliza la función de activación **sigmoid**, ideal para generar resultados en el rango [0, 1], típicamente usados para tareas de clasificación, para este caso de uso se utiliza por que las señales de activacion de los motores es digital.

#### 2. Parámetros Importantes
- **`input_shape`**:
  - Define la cantidad de características de entrada que el modelo debe esperar (por ejemplo, 4 si hay 4 columnas en `X_train`).
- **`Dense(8, activation='relu')`**:
  - Una capa completamente conectada con 8 neuronas.
  - **ReLU** ayuda al modelo a aprender patrones no lineales.
- **`Dense(4, activation='sigmoid')`**:
  - Una capa completamente conectada con 4 neuronas (una para cada salida esperada, como IN1, IN2, IN3, IN4).
  - **Sigmoid** asegura que las salidas estén en el rango de 0 a 1.

#### 3. Manejar Errores
```python
except Exception as e:
    print(f"Error al crear el modelo de la red neuronal: {e}")
    exit()
```
- **¿Qué hace esto?**
  - Si ocurre algún problema durante la creación del modelo (por ejemplo, si los datos de entrada no están bien definidos), muestra un mensaje de error y detiene el programa.



In [21]:
# Crear el modelo de la red neuronal
try:
    input_shape = (X_train.shape[1],)  # Determinar automáticamente el número de características de entrada
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=input_shape),        # Entrada con la forma del dataset
        tf.keras.layers.Dense(8, activation='relu'),  # Capa oculta con 8 neuronas
        tf.keras.layers.Dense(4, activation='sigmoid') # Cuatro salidas (para IN1, IN2, IN3, IN4)
    ])
except Exception as e:
    print(f"Error al crear el modelo de la red neuronal: {e}")
    exit()

# Explicación

Este bloque de código realiza dos pasos esenciales en el desarrollo de una red neuronal: compilar el modelo y entrenarlo con los datos.

### Código Explicado

#### 1. Compilar el Modelo
```python
try:
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
except Exception as e:
    print(f"Error al compilar el modelo: {e}")
    exit()
```
- **¿Qué significa esto?**
  - **Compilar el modelo** prepara la red neuronal para ser entrenada, definiendo:
    1. **Optimizador (`optimizer='adam'`)**: Controla cómo el modelo ajusta sus parámetros para mejorar el rendimiento.
    2. **Función de pérdida (`loss='binary_crossentropy'`)**: Calcula qué tan lejos están las predicciones del modelo de los valores reales. Es adecuada para problemas con salidas binarias (0 o 1).
    3. **Métrica (`metrics=['accuracy']`)**: Supervisa la precisión durante el entrenamiento para evaluar el desempeño del modelo.

- **Manejo de Errores**:
  - Si algo falla al compilar (por ejemplo, configuraciones incompatibles), muestra un mensaje de error y detiene el programa.

#### 2. Entrenar el Modelo
```python
try:
    history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2, verbose=1)
    print("Entrenamiento completado.")
except Exception as e:
    print(f"Error durante el entrenamiento del modelo: {e}")
    exit()
```
- **¿Qué significa esto?**
  - **Entrenar el modelo** significa enseñarle a predecir correctamente usando los datos de entrenamiento.
  - Los parámetros importantes son:
    1. **`epochs=50`**: El número de veces que el modelo verá todos los datos durante el entrenamiento. Más épocas pueden mejorar el aprendizaje, pero también aumentar el riesgo de "sobreajuste".
    2. **`batch_size=8`**: El modelo procesa los datos en lotes de 8 elementos para mejorar la eficiencia.
    3. **`validation_split=0.2`**: Reserva el 20% de los datos de entrenamiento para validación, ayudando a medir el rendimiento del modelo en datos no vistos.
    4. **`verbose=1`**: Muestra el progreso del entrenamiento en la consola.


In [22]:
# Compilar el modelo
try:
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
except Exception as e:
    print(f"Error al compilar el modelo: {e}")
    exit()
# Entrenar el modelo
try:
    history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2, verbose=1)
    print("Entrenamiento completado.")
except Exception as e:
    print(f"Error durante el entrenamiento del modelo: {e}")
    exit()

Epoch 1/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1125 - loss: 0.7021 - val_accuracy: 0.1789 - val_loss: 0.6605
Epoch 2/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2201 - loss: 0.6561 - val_accuracy: 0.2174 - val_loss: 0.6474
Epoch 3/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2199 - loss: 0.6450 - val_accuracy: 0.2358 - val_loss: 0.6415
Epoch 4/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2327 - loss: 0.6428 - val_accuracy: 0.2241 - val_loss: 0.6359
Epoch 5/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2167 - loss: 0.6323 - val_accuracy: 0.2408 - val_loss: 0.6301
Epoch 6/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2235 - loss: 0.6233 - val_accuracy: 0.2542 - val_loss: 0.6237
Epoch 7/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2434 - loss: 0.6227 - val_accuracy: 0.2324 - val_loss: 0.6172
Epoch 8/50
299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2384 - loss: 0.6152 - val_accuracy: 0.

# Explicación

Este bloque de código evalúa el desempeño del modelo de red neuronal utilizando los datos de prueba. Esto ayuda a determinar qué tan bien funciona el modelo con datos que no ha visto antes.


#### Evaluar el Modelo
```python
try:
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f'Pérdida de prueba: {test_loss:.4f}, Precisión de prueba: {test_accuracy:.4f}')
except Exception as e:
    print(f"Error al evaluar el modelo: {e}")
    exit()
```
- **¿Qué significa esto?**
  - La función `model.evaluate()` calcula:
    1. **Pérdida de prueba (`test_loss`)**: Una medida que indica qué tan lejos están las predicciones del modelo de los valores reales. Un valor más bajo es mejor.
    2. **Precisión de prueba (`test_accuracy`)**: Indica el porcentaje de predicciones correctas hechas por el modelo en el conjunto de prueba. Un valor más alto es mejor.

- **Parámetros Importantes**:
  - **`X_test` y `y_test`**: Los datos de prueba que el modelo no ha visto durante el entrenamiento, usados para evaluar su desempeño.
  - **`verbose=2`**: Muestra un nivel intermedio de detalles sobre el proceso de evaluación.

- **Manejo de Errores**:
  - Si algo falla durante la evaluación (por ejemplo, datos mal estructurados o un modelo mal configurado), se captura la excepción, se imprime un mensaje de error y el programa se detiene.



In [23]:
# Evaluar el modelo con los datos de prueba
try:
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f'Pérdida de prueba: {test_loss:.4f}, Precisión de prueba: {test_accuracy:.4f}')
except Exception as e:
    print(f"Error al evaluar el modelo: {e}")
    exit()

24/24 - 0s - 2ms/step - accuracy: 0.4332 - loss: 0.5771
Pérdida de prueba: 0.5771, Precisión de prueba: 0.4332


# Explicación 

Este bloque de código genera un archivo de cabecera (`.h`) que contiene los pesos y sesgos del modelo de red neuronal en formato C/C++. Este archivo es útil para integrar los parámetros del modelo en aplicaciones de hardware o sistemas embebidos.

#### 1. Definir la Ruta del Archivo de Salida
```python
carpeta_de_salida = Carpeta_de_entrenamiento.replace('datos_iniciales_del_sistema', 'salida_archivo_con_pesos_red_neuronal')
ruta_h = os.path.join(carpeta_de_salida, 'pesos_red_neuronal.h')
```
- **¿Qué significa esto?**
  - Se define una carpeta para almacenar el archivo generado, reemplazando el directorio base `datos_iniciales_del_sistema` por `salida_archivo_con_pesos_red_neuronal`.
  - Se construye la ruta completa para el archivo de salida llamado `pesos_red_neuronal.h`.


- **¿Qué hace esto?**
  1. Abre (o crea) un archivo `.h` en la ubicación especificada.
  2. Escribe las declaraciones necesarias para definir constantes de pesos y sesgos para cada capa del modelo.
  3. Cada capa contiene:
     - **Pesos**: Representados como un arreglo unidimensional en C.
     - **Sesgos**: También representados como un arreglo en C.
  4. Utiliza bucles para recorrer todas las capas del modelo y extraer los valores.
  5. Maneja errores que puedan ocurrir al obtener los pesos y sesgos de una capa específica.

- **Estructura del Archivo Generado**:
  - Comienza con definiciones para evitar múltiples inclusiones (`#ifndef ... #define ...`).
  - Contiene los pesos y sesgos para cada capa en formato de arreglo estático de punto flotante.
  - Finaliza con una directiva para cerrar el bloque de inclusión.



In [25]:
# Generar el archivo de cabecera (.h) para los pesos y sesgos
carpeta_de_salida=Carpeta_de_entrenamiento.replace('datos_iniciales_del_sistema','salida_archivo_con_pesos_red_neuronal')


ruta_h = os.path.join(carpeta_de_salida, 'pesos_red_neuronal.h')

try:
    with open(ruta_h, 'w') as archivo_h:
        archivo_h.write("#ifndef PESOS_RED_NEURONAL_H\n")
        archivo_h.write("#define PESOS_RED_NEURONAL_H\n\n")
        
        for i, layer in enumerate(model.layers):
            try:
                weights, biases = layer.get_weights()
                
                archivo_h.write(f"// Pesos de la capa {i}\n")
                archivo_h.write(f"static const float PESOS_CAPA_{i}[] = {{\n")
                
                flat_weights = weights.flatten()
                archivo_h.write(", ".join([f"{w:.6f}" for w in flat_weights]))
                archivo_h.write("\n};\n\n")
                
                archivo_h.write(f"// Sesgos de la capa {i}\n")
                archivo_h.write(f"static const float SESGOS_CAPA_{i}[] = {{\n")
                archivo_h.write(", ".join([f"{b:.6f}" for b in biases]))
                archivo_h.write("\n};\n\n")
                
            except Exception as e:
                print(f"Error al obtener los pesos y sesgos de la capa {i}: {e}")
        
        archivo_h.write("#endif // PESOS_RED_NEURONAL_H\n")
        print(f"Archivo de pesos generado exitosamente en: {ruta_h}")
except Exception as e:
    print(f"Error al escribir el archivo de pesos: {e}")
    exit()


Archivo de pesos generado exitosamente en: c:\Users\dgome\OneDrive\SIN RECUPERAR\Sql\Documentos\SistemaIot\Periferico\modelos_entrenamiento\vehiculo_autonomo\salida_archivo_con_pesos_red_neuronal\pesos_red_neuronal.h
